# Toonify yourself!

Please ensure that you're using a GPU runtime

First some setup:

In [ ]:
%tensorflow_version 1.x

In [ ]:
!git clone https://github.com/justinpinkney/stylegan2
%cd stylegan2
!nvcc test_nvcc.cu -o test_nvcc -run

In [ ]:
!mkdir raw
!mkdir aligned
!mkdir generated

## Upload your own photos

Upload your photos to `raw/`. These don't need to be aligned as we'll use a face detector to grab all the faces and transform them into the correct format. One note of caution is that you'll need a pretty high-resolution picture of a face to get a sharp result (the final face crop is resized to 1024x1024 pixels)

We'll grab a example image from the internet to work with.

The basic process is:
- Extract faces and align the images
- Project the images (i.e. find the latent code)
- Toonify the images (i.e. use the latent code with the toon model)

Results will be placed in the stylegan2/generated folder

In [ ]:
# 적용시킬 얼굴
!wget http://monthly.chosun.com/up_fd/Mdaily/2019-05/bimg_thumb/%EB%A7%88%EB%8F%99%EC%84%9D.jpg -O raw/example.jpg

# 문제 시작
방법1 - invail 뭐시기 오류 발생 ; 크기가 너무 커서 url에 있는 plk파일 못 받아오는 건가??
방법2 - 그럼 피클 파일을 그냥 colab파일에 업로드해서 열어보자! -> module문제발생

방법1

In [ ]:
import pretrained_networks

# use my copy of the blended model to save Doron's download bandwidth
# get the original here https://mega.nz/folder/OtllzJwa#C947mCCdEfMCRTWnDcs4qw
# 원래 있던 blended_url
# blended_url = "https://drive.google.com/uc?id=1H73TfV5gQ9ot7slSed_l-lim9X7pMRiU"
# 사용할 지브리 blended_url
blended_url = "https://drive.google.com/file/d/1-l62KDZWBr8hdh6Vnf9-l-Ts3wUs5IO_/view?usp=sharing"
ffhq_url = "http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-ffhq-config-f.pkl"

 _, _, Gs_blended = pretrained_networks.load_networks(blended_url)
 _, _, Gs = pretrained_networks.load_networks(ffhq_url)

In [ ]:
import pretrained_networks
import pickle
# from utils.torch_utils import select_device, time_synchronized
import torch_utils

ffhq_url = "http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-ffhq-config-f.pkl"

# 여기서 문제 발생.. 으엑
with open('/content/network-snapshot-000000.pkl', 'rb') as f:
    loaded = pickle.load(f, encoding="latin1") 

 _, _, Gs = pretrained_networks.load_networks(ffhq_url)

In [ ]:
!python align_images.py raw aligned

In [ ]:
!python project_images.py --num-steps 500 aligned generated

In [ ]:
import numpy as np
from PIL import Image
import dnnlib
import dnnlib.tflib as tflib
from pathlib import Path

latent_dir = Path("generated")
latents = latent_dir.glob("*.npy")
for latent_file in latents:
  latent = np.load(latent_file)
  latent = np.expand_dims(latent,axis=0)
  synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=False), minibatch_size=8)
  images = Gs_blended.components.synthesis.run(latent, randomize_noise=False, **synthesis_kwargs)
  Image.fromarray(images.transpose((0,2,3,1))[0], 'RGB').save(latent_file.parent / (f"{latent_file.stem}-toon.jpg"))


In [ ]:
from IPython.display import Image 
embedded = Image(filename="generated/example_01.png", width=256)
display(embedded)
tooned = Image(filename="generated/example_01-toon.jpg", width=256)
display(tooned)